In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import config
import helpers
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
from itertools import islice
#filename = './Auswertung_Susanne.xlsx'
#filename = './wearcheck_Jenbacher S Oil.xlsx'
#filename = './test.xlsx'
filename = './csv_converted.xlsx'

In [3]:
fields = ['Unique Code','General Description','Oil/Fluid Long Name','Unit Hours','Oil/Fluid Hours'] # auf die benötigten Felder beschränken

In [4]:

wb = load_workbook(filename)
wbsheets = wb.sheetnames
ws0 = wb[wbsheets[0]]
data = ws0.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
xdf = pd.DataFrame(data, index=idx, columns=cols)
#xdf = pd.DataFrame(data, columns=cols)
xdf = xdf[fields]


In [5]:
#xdf = pd.read_excel(filename)[fields]

In [6]:
xdf.head(2)

,Unique Code,General Description,Oil/Fluid Long Name,Unit Hours,Oil/Fluid Hours
4301368,1445343 Z588,BGA Günther I - Container Nord,Mobil Pegasus 605 Ultra,220.0,220.0
4301370,1445343 Z588,BGA Günther I - Container Nord,Mobil Pegasus 605 Ultra,388.0,168.0


In [7]:
corr_xdf = xdf.applymap(helpers.corr) # correct the contents
corr_xdf

,Unique Code,General Description,Oil/Fluid Long Name,Unit Hours,Oil/Fluid Hours
4301368,1445343 Z588,BGA Günther I - Container Nord,Mobil Pegasus 605 Ultra,220.0,220.0
4301370,1445343 Z588,BGA Günther I - Container Nord,Mobil Pegasus 605 Ultra,388.0,168.0
3289304,1164601 T749,Spie Fujitsu Augsburg,Fuchs Titan Ganymet Ultra,9684.0,3948.0
3625990,1320161 V040,BGA Menz II,Total Nateria MP 40,4030.0,1980.0
4212154,1313076 U819,Klärschlammvergasung Koblenz,Fuchs Titan Ganymet Ultra,477.0,NaN
...,...,...,...,...,...
3640619,1166070 T787,BHKW Donautal Geflügel II,Mobil Pegasus 705,14277.0,1283.0
3640621,1166070 T787,BHKW Donautal Geflügel II,Mobil Pegasus 705,13693.0,698.0
3818292,1166070 T787,BHKW Donautal Geflügel II,Mobil Pegasus 705,18387.0,530.0
3640614,1166074 T787,BHKW Donautal Geflügel I,Mobil Pegasus 705,12542.0,2000.0


In [8]:
result = {
    'Zeilen': [],
    'Eindeutige Motoren': [],
    'gültige Motoren': [],
    'Kumulierte Öl Stunden': [],
    'Mittlere Öl Stunden pro gültigem Motor': [],
    }
columns = list(corr_xdf['Oil/Fluid Long Name'].unique())
print(columns)

['Mobil Pegasus 605 Ultra', 'Fuchs Titan Ganymet Ultra', 'Total Nateria MP 40', 'keine Angabe', 'Tectrol MethaFlexx HC premium', 'Mobil Pegasus 705', 'Petro-Canada Sentron LD 5000', 'Addinol GMO MG 40 Extra LA', 'Tectrol MethaFlexx Premium', 'Tectrol MethaFlexx premium', 'Mobil Pegasus 710', 'Mobil Pegasus 610', 'Addinol Gas Engine Oil NG 40', 'Fuchs Ganymet 40', 'Haertol Frostox W35/65', 'Q8 Mahler G5 SAE 40']


In [9]:
for oil_name in columns:
    new_xdf = corr_xdf[corr_xdf['Oil/Fluid Long Name'] == oil_name] # nur die Zeilen mit 'oil_name' ausfiltern
    uniquecodes = new_xdf['Unique Code'].unique() # Alle Unique Codes der Motoren auslesen - nur unterschiedliche codes kommen in die Liste
    No_of_Engines = len(uniquecodes)
    result['Zeilen'].append(new_xdf.shape[0])
    result['Eindeutige Motoren'].append(No_of_Engines)
    if new_xdf.shape[0] == 0: #no enties found
        raise ValueError(f"kein Öl mit Namen '{oil_name}' in '{os.path.basename(filename)}' gefunden.")
    oil_sum = 0
    count_sum = 0
    for u in uniquecodes:
        df = new_xdf[new_xdf['Unique Code'] == u].sort_values(by = ['Unit Hours'],ascending=[True])
        try: #try to calculate oil running hours
            oil_age = df['Unit Hours'].max() - df['Unit Hours'].min() + df.iloc[0]['Oil/Fluid Hours']
            if oil_age == oil_age: #this is a trick to check for NAN
                oil_sum += oil_age
                count_sum += 1
        except Exception:
            pass # do nothing on entry rows with missing parameters.
    result['gültige Motoren'].append(count_sum)
    result['Kumulierte Öl Stunden'].append(int(f"{oil_sum:0.0f}"))
    result['Mittlere Öl Stunden pro gültigem Motor'].append(int(f"{oil_sum / (count_sum or 1):0.0f}"))

In [10]:
result[' '] = ['']*len(columns)
result['Datum'] = [pd.Timestamp.now()] + ['']* (len(columns)-1)
result['Datei'] = [os.path.basename(filename)]  + ['']* (len(columns)-1)

result;

In [11]:
rdf = pd.DataFrame.from_dict(result, columns=columns, orient='index')
#logging.info(f"Exporting Zusammenfassung to {config.zoutfile}.")
#rdf.to_excel(config.zoutfile)
#print(rdf)
#os.startfile(config.zoutfile)

In [ ]:
wb.create_sheet('Zusammenfassung')
wbsheets = wb.sheetnames
ws = wb[wbsheets[-1]]
for r in dataframe_to_rows(rdf, index=True, header=True):
    ws.append(r)
wb.save(filename)
wb.close()

In [ ]:
import pandas as pd
t = {
    'A':[1,2,3,4],
    'B':[1,2,3,4],
    'C':[1,2,3,4],
    'D':[1,2,3,4],
    'E':[1,2,3,4],
    ' ':['','','',''],
    'Datum':['1.1.2122','','',''],
    'Datei':['jhgfjhg','','','']
}
tdf = pd.DataFrame.from_dict(t, columns=['Eins','Zwei','Drei','Vier'], orient='index')
#tdf.to_excel(config.zoutfile)

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
wb = load_workbook('./test.xlsx')
print(wb.sheetnames)
wb.create_sheet('Zusammenfassung')
sheets = wb.sheetnames
print(sheets)
ws = wb[sheets[-1]]
print(ws.title)
for r in dataframe_to_rows(tdf, index=True, header=True):
    ws.append(r)
wb.save('./test.xlsx')
wb.close()